# HW 1 for Projects in ML and AI

In [1]:
# installing dependencyes

!pip install -q kaggle

In [8]:
# mount drive and set up kaggle API

from google.colab import drive
drive.mount('/content/drive')

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/.kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# install dataset and unzip

!rm -r dataset

!kaggle datasets download -d saumitgp/occupancy-detection-dataset
!mkdir dataset
!unzip occupancy-detection-dataset.zip -d dataset

  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 71.1MB/s]
Archive:  occupancy-detection-dataset.zip
  inflating: dataset/OccupancyData/DataTest.csv  
  inflating: dataset/OccupancyData/DataTraining.csv  


In [51]:
df.isna().any()

date             False
Temperature      False
Humidity         False
Light            False
CO2              False
HumidityRatio    False
Occupancy        False
dtype: bool

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

# the balance is weird, I will merge the train and test files first and split them to 8:2
df = pd.concat([pd.read_csv('dataset/OccupancyData/DataTraining.csv',index_col=[0]),
                pd.read_csv('dataset/OccupancyData/DataTest.csv',index_col=[0])])
X = df.drop(columns=['date'])
y = X.pop('Occupancy')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=0)


Task 1 (10 points): Describe a machine learning problem that you would like to solve using 
Logistic Regression. Clearly state why Logistic regression is the best choice for solving this 
problem.

## Task 1

The problem of detecting occupancy in a room (classificaion) can be solved using a classification algorithm and training on various features, such as light, co2, temperature, and humidity of a room

Task 2 (20 points): Pick a publicly available dataset (except The Titanic Dataset because we 
already discussed that in class) that you will use to solve this problem. You must provide a link 
to the dataset and perform necessary Exploratory Data Analysis (EDA). Clearly demonstrate the 
steps you follow for your EDA with a justification of why these were required. For example, if 
the dataset has lot of missing values, then why did you use a specific technique when handling 
missing data. This task may include data visualization (Check this link : 
https://www.geeksforgeeks.org/top-8-python-libraries-for-data-visualization/).

## Task 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# the balance is weird, I will merge the train and test files first and split them to 8:2
df = pd.concat([pd.read_csv('dataset/OccupancyData/DataTraining.csv',index_col=[0]),
                pd.read_csv('dataset/OccupancyData/DataTest.csv',index_col=[0])])
X = df.drop(columns=['date'])
y = X.pop('Occupancy')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=0)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

clf = LogisticRegression()
clf.fit(X_train,y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

Task 3 (35 points): Implement, Logistic Regression in this step. Clearly write your cost function 
and derivatives before implementing gradient descent. Do not use any built-in packages for this 
step. You can use the vectorization techniques demonstrated in class. Implement any 2 variants 
of gradient descent in their original form. (Refer to the research paper discussed in class).

## Task 3

In [ ]:
import numpy as np 

class LogisticRegression():
    def __init__(self):
        # initialize as random zeros. Using 100 weights, but for each dataset with N features we can use the first N weights and biases
        self.w = np.zeroes(100)
        self.b = np.zeros(100)
    def 

Task 4 (35 points): Pick any 2 optimization algorithms that are used to optimize the ‘vanilla’
gradient descent. Implement both. You may implement these algorithms yourself OR use a 
package. In your conclusion, compare both optimization techniques/algorithms with respect to 
the results you achieve. Also compare these results with the original implementation of 
gradient descent (Task 3 above). Describe why or why not should we use optimization 
algorithms for the task at hand

## Task 4

In [ ]:
from sklearn.linear_model import LogisticRegression